In [1]:
import numpy

In [2]:
import tensorflow

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
import main, pickle

In [6]:
random_board_pairs = []
with open("2021-09-18-mcts-v3-boards13-random.pickle", "rb") as f:
    try:
        while True:
            random_board_pairs.append(pickle.load(f))
    except EOFError:
        pass

In [7]:
len(random_board_pairs)

20156

In [4]:
game_board_pairs = []
with open("2021-10-12-mcts-v3-boards14-games.pickle", "rb") as f:
    try:
        while True:
            game_board_pairs.append(pickle.load(f))
    except EOFError:
        pass

In [5]:
len(game_board_pairs)

21652

In [8]:
random_inputs = []
random_outputs = []
for p in random_board_pairs:
    random_inputs.append(p[0].get_flipped_3d_array())
    random_outputs.append(float(p[1]))

In [9]:
game_inputs = []
game_outputs = []
for p in game_board_pairs:
    game_inputs.append(p[0].get_flipped_3d_array())
    game_outputs.append(float(p[1]))

In [10]:
import numpy as np

In [11]:
game_inputs = np.array(game_inputs)
random_inputs = np.array(random_inputs)
game_outputs = np.array(game_outputs)
random_outputs = np.array(random_outputs)

In [12]:
random_mark = int(len(random_outputs)*0.8)

In [13]:
game_mark = int(len(game_outputs)*0.8)

In [14]:
print(random_mark, game_mark)

16124 17321


In [15]:
train_inputs = np.concatenate((random_inputs[0:random_mark] , game_inputs[0:game_mark] 
                               , random_inputs[random_mark:] ,  game_inputs[game_mark:])) 

In [16]:
train_outputs = np.concatenate(( random_outputs[0:random_mark],
                                 game_outputs[0:game_mark],
                                random_outputs[random_mark:] ,
                                game_outputs[game_mark:] ))

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [18]:
model2 = keras.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 15, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [19]:
model2.add(layers.Flatten())
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(1, activation='relu'))

In [20]:
model2.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(0.001))

In [21]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 13, 32)        608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 11, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 10560)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               2703616   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 2,722,977
Trainable params: 2,722,977
Non-trainable params: 0
_________________________________________________________________


In [22]:
model2.fit(train_inputs, train_outputs, epochs=10, validation_split = 0.2)

Epoch 1/10
1046/1046 [==============================] - 25s 23ms/step - loss: 0.1010 - val_loss: 0.0582
Epoch 2/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0552 - val_loss: 0.0536
Epoch 3/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0505 - val_loss: 0.0580
Epoch 4/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0477 - val_loss: 0.0495
Epoch 5/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0452 - val_loss: 0.0488
Epoch 6/10
1046/1046 [==============================] - 23s 22ms/step - loss: 0.0429 - val_loss: 0.0498
Epoch 7/10
1046/1046 [==============================] - 25s 24ms/step - loss: 0.0406 - val_loss: 0.0517
Epoch 8/10
1046/1046 [==============================] - 29s 28ms/step - loss: 0.0387 - val_loss: 0.0504
Epoch 9/10
1046/1046 [==============================] - 31s 29ms/step - loss: 0.0370 - val_loss: 0.0484
Epoch 10/10
1046/1046 [==============================] - 30s 28m

When training v3 (which did better than v2), CV loss was about 0.029-30. 

When training v4 (which wasn't an improvement and in fact might have done worse than v3), CV loss was about 0.028-29

When training v5 (which didn't do better than neural net v3), CV loss was about 0.048
                                                                                 
Now v7's CV loss has plateaued at 0.049. Hmm. Maybe that's just a reflection of the fact that v5 and v7's data came from MCTS? I hope the data isn't messy

MCTS5 was supposed to generate less messy data, but I nixed that because it lost to MCTS3

(Note: I trained options for a v6, but never saved one bc the CV lossses didn't look better than for v5, and it was the same data anyway)

In [23]:
model2.save('saved_model/2021-10-26-v7-model')

INFO:tensorflow:Assets written to: saved_model/2021-10-26-v7-model/assets
